In [ ]:
from nbdev import *

In [ ]:
%nbdev_default_export dcae

Cells will be exported to pacmel_mining_use_case.dcae,
unless a different module is specified after an export flag: `%nbdev_export special.module`


In [ ]:
%nbdev_hide
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2

# DCAE: Deep Convolutional Autoencoder

> This notebook tries to apply the ideas of the paper [TimeCluster](https://link.springer.com/article/10.1007/s00371-019-01673-y),
with regard to the application of DCAEs for compressing multivariate time series data.

In [ ]:
%nbdev_export
from fastcore import test
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv1D, MaxPool1D, Reshape, UpSampling1D, InputLayer

In [ ]:
%nbdev_hide
from pacmel_mining_use_case.load import *
from tensorflow.keras.optimizers import Adam
import wandb
from wandb.keras import WandbCallback
from yaml import load, FullLoader
from fastcore.utils import Path
from datetime import datetime
import pickle

For the experiment tracking and hyperparameter we will use the tool **Weights & Biases**. 

Before running this notebook, make sure you run in a terminal `wandb login [API_KEY]`. You can see your API_KEY in the settings of your wandb account.

In [ ]:
%nbdev_hide
run_dcae = wandb.init(project="timecluster-extension",
                      job_type='train_DCAE',
                      allow_val_change=True,
                      resume=False)
config = wandb.config  # Object for storing hyperparameters

wandb: Currently logged in as: vrodriguezf (use `wandb login --relogin` to force relogin)


## Loading the dataset

To load the dataset we will download a specific dataset artifact from the collection of artifacts
stored in the weights and biases (wandb) project associated to this experiment.

In [ ]:
artifact_name_and_version = 'JNK:v7'

In [ ]:
ds_artifact = run_dcae.use_artifact(artifact_name_and_version)

In [ ]:
# parameters (uncomment to override the yaml file)
config.update(
    {
          'ds_artifact_type': ds_artifact.type,
          'ds_artifact_name': ds_artifact.name,
          'ds_artifact_digest': ds_artifact.digest
    }, 
    allow_val_change=True)
ds_artifact.type, ds_artifact.name, ds_artifact.digest

('dataset', 'JNK:v7', '3f846b69e5c7e6df6683460323dcba0b')

The artifact must have been logged from a `TSArtifact` object. IF that's true, the metadata of the downloaded artifact will contain all the necessary information to recover the dataframe containing the time series.

In [ ]:
ds_artifact.metadata

{'TS': {'ed': '2019-06-30 00:00:00',
  'sd': '2019-06-01 00:00:00',
  'freq': '<5 * Seconds>',
  'hash': '-2568758837685750672',
  'vars': ['RCD_AverageThree-phaseCurrent',
   'LCD_AverageThree-phaseCurrent',
   'LP_AverageThree-phaseCurrent',
   'LHD_LeftHaulageDrive(tractor)Temperature(gearbox)',
   'RHD_RightHaulageDrive(tractor)Temperature(gearbox)',
   'LA_LeftArmTemperature',
   'RA_RightArmTemperature',
   'SM_DailyRouteOfTheShearer',
   'SM_TotalRoute',
   'LHD_EngineCurrent',
   'RHD_EngineCurrent',
   'RCD_BearingTemperature',
   'SM_ShearerSpeed',
   'SM_ShearerLocation',
   'SM_ShearerMoveInLeft',
   'SM_ShearerMoveInRight'],
  'n_vars': 16,
  'created': 'from-df',
  'n_samples': 501121,
  'normalization': {'stds': {'SM_TotalRoute': 25.04366490593873,
    'SM_ShearerSpeed': 2.9198292690278103,
    'LHD_EngineCurrent': 21.93831294062176,
    'RHD_EngineCurrent': 21.21578937550054,
    'SM_ShearerLocation': 78.26614419252932,
    'SM_ShearerMoveInLeft': 0.2554890106589118,
  

In [ ]:
df = ds_artifact.to_df()

wandb: Downloading large artifact JNK:v7, 65.00MB. 1 files... Done. 4.9s


The data that will be used in the rest of the notebook will be stored in the dataframe `df`

In [ ]:
df

,RCD_AverageThree-phaseCurrent,LCD_AverageThree-phaseCurrent,LP_AverageThree-phaseCurrent,LHD_LeftHaulageDrive(tractor)Temperature(gearbox),RHD_RightHaulageDrive(tractor)Temperature(gearbox),LA_LeftArmTemperature,RA_RightArmTemperature,SM_DailyRouteOfTheShearer,SM_TotalRoute,LHD_EngineCurrent,RHD_EngineCurrent,RCD_BearingTemperature,SM_ShearerSpeed,SM_ShearerLocation,SM_ShearerMoveInLeft,SM_ShearerMoveInRight
TIMESTAMP,,,,,,,,,,,,,,,,
2019-06-01 00:00:00,-0.971566,-0.958893,-0.714756,-2.506141,-0.22915,-3.602263,-3.374381,-1.065309,-6.196249,-0.591877,-0.590393,-3.528427,-0.391898,0.807398,-0.284494,-0.346266
2019-06-01 00:00:05,-0.971566,-0.958893,-0.714756,-2.506141,-0.22915,-3.602263,-3.374381,-1.065309,-6.196249,-0.591877,-0.590393,-3.528427,-0.391898,0.807398,-0.284494,-0.346266
2019-06-01 00:00:10,-0.971566,-0.958893,-0.714756,-2.506141,-0.22915,-3.602263,-3.374381,-1.065309,-6.196249,-0.591877,-0.590393,-3.528427,-0.391898,0.807398,-0.284494,-0.346266
2019-06-01 00:00:15,-0.971566,-0.958893,-0.714756,-2.506141,-0.22915,-3.602263,-3.374381,-1.065309,-6.196249,-0.591877,-0.590393,-3.528427,-0.391898,0.807398,-0.284494,-0.346266
2019-06-01 00:00:20,-0.971566,-0.958893,-0.714756,-2.506141,-0.22915,-3.602263,-3.374381,-1.065309,-6.196249,-0.591877,-0.590393,-3.528427,-0.391898,0.807398,-0.284494,-0.346266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-29 23:59:40,-0.639781,-0.669682,-0.714756,0.787827,-0.22915,0.415994,0.293725,-0.333925,1.474354,-0.591877,-0.590393,0.253327,-0.391898,0.781844,-0.284494,-0.346266
2019-06-29 23:59:45,-0.639781,-0.669682,-0.714756,0.787827,-0.22915,0.415994,0.293725,-0.333925,1.474354,-0.591877,-0.590393,0.253327,-0.391898,0.781844,-0.284494,-0.346266
2019-06-29 23:59:50,-0.639781,-0.669682,-0.714756,0.787827,-0.22915,0.415994,0.293725,-0.333925,1.474354,-0.591877,-0.590393,0.253327,-0.391898,0.781844,-0.284494,-0.346266


## Train

### Sliding window features

Define a continuous multivariate time-series data $D$ of dimension $d$ with $n$ time-steps, $D = X_1,X_2,\dots,X_n$ , where each $X_i = \{x_i^1,\dots,x_i^d\}$ . Let $w$ be the window width, $s$ the stride, and $t$ the start time of a sliding window in the data.

Define a new matrix $Z_k$ where each row is a vector of size $w$ of data extracted from the $k^{th}$ dimension.

\begin{aligned}&Z_k(w,s,t)\\&\quad =\begin{bmatrix} x_{t}^k&\quad x_{t+1}^k&\quad \dots&\quad x_{t+w-1}^k \\ x_{t+s}^k&\quad x_{t+s+1}^k&\quad \dots&\quad x_{t+s+w-1}^k \\ \vdots&\quad \vdots&\quad \ddots&\quad \vdots \\ x_{t+(r-1)s}^k&\quad x_{t+(r-1)s+1}^k&\quad \dots&\quad x_{t+(r-1)s+w-1}^k \end{bmatrix} \end{aligned}

where $r$ is the number of desired rows, and $t+(r-1)s+w-1 \le n$


$Z$ is a $w \times s \times t$ matrix. The first step consists in slicing the original multivariate time series into slices of shape ($w \times d$), as shown in this figure from the paper.
<img src="https://i.imgur.com/R9Fx8uO.png" style="width:800px;height:400px"/>

In [ ]:
# parameters (uncomment to override the yaml file)
config.update({
    'w': 48,
    'stride': 1,
    't': 0  # TODO: Not supported yet
    }, allow_val_change=True)

In [ ]:
test.equals(config.w % 12, 0)

True

The sliced data must be converted to a numpy array with shape $(n \times w \times d)$, where $n$ is the length of the time series, $w$ is the window size and $d$ is the number of dimensions in the time series. 

In [ ]:
%nbdev_export utils
def df_slicer(df, w, s=1, padding=False, padding_value=0, return_as='ndarray'):
    "Transform a numeric dataframe `df` into slices (np arrays) of `w` \
    rows and the same number of columns than the original dataframe. The \
    distance between each slice is given by the stride `s`. If `padding` is \
    equals to True, the last slices which have less than `w` points are filled \
    with the value marked in the argument `padding_value`. Otherwise, those \
    slices are removed from the result. TODO: the argument `return_as` is currently \
    not used"
    aux = [df.iloc[x:x+w] for x in range(0, len(df), s)]
    if padding:
        with_padding = [x.append(pd.DataFrame(
            np.full((w - len(x), len(df.columns)), padding_value),
            columns=df.columns.values)) if len(x) < w else x for x in aux]
    else:
        with_padding = [x for x in aux if len(x) == w]
    return np.rollaxis(np.dstack([x.values for x in with_padding]), -1)

In [ ]:
input_data = df_slicer(df, w=config.w, s=config.stride)
len(input_data)

501074

In [ ]:
input_data[0][0]

array([-0.97156555, -0.95889268, -0.71475646, -2.50614084, -0.22915015,
       -3.6022626 , -3.37438141, -1.0653093 , -6.19624895, -0.59187731,
       -0.59039292, -3.52842662, -0.39189815,  0.8073983 , -0.28449447,
       -0.34626564])

Test the number of slices and the size of each slice

In [ ]:
expected_nwindows = (int)((len(df) - config.w)/config.stride + 1)
expected = [(config.w, len(df.columns))]*expected_nwindows
actual = [x.shape for x in input_data]
test.all_equal(expected, actual)

True

## Extract important features from the multivariate time series data through Deep Convolutional Autoencoders


Deep Convolutional Auto Encoders (DCAE) is a powerful method for learning high-level and mid-level abstractions from low-level raw data. It has the ability to extract features from complex and large time-series in an unsupervised manner. This is useful to overcome the complexity of multivariate time-series.

Compared to the conventional auto-encoder, DCAE has fewer parameters than the conventional auto-encoder which means less training time. Also, DCAE uses local information to reconstruct the signal while conventional auto-encoders utilize fully connected layers to globally do the reconstruction. DCAE is an unsupervised model for representation learning which maps inputs into a new representation space. It has two main parts which are the encoding part that is used to project the data into a set of feature spaces and the decoding part that reconstructs the original data. The latent space representation is the space where the data lie in the bottleneck layers.

The loss function of the DCAE is defined as the error between the input and the output. DCAE aims to find a code for each input by minimizing the mean squared error (MSE) between its input (original data) and output (reconstructed data). The MSE is used which assists to minimize the loss; thus, the network is forced to learn a low-dimensional representation of the input.

We will implement the DCAE of the paper [TimeCluster](https://link.springer.com/article/10.1007/s00371-019-01673-y), whose architecture is shown in the table below:

![](https://i.imgur.com/3EjuAfQ.png)

Note that, in the paper, the input shape is $60 \times 3$, due to multivariate time series has 3 variables and the window size is 60. Generally, the size of the input/output of the autoencoder will depend on the shape of each slice obtained in the previos step. The number of latent features to be discovered is $60$ in the table above, but we can consider this as a free hyperparameter $\delta$. Also, according to the paper: "*The number of feature maps, size of filter and depth of the model are set based on the reconstruction error on validation set.*". Thus, we must provide flexibility in the creation of the DCAE in terms of these hyperparameters.º

In case you are not using a config file, you can also uncomment the following cell and define the hyperparameters in the fly

In [ ]:
%nbdev_hide
config.update({
        'lr': 0.0009044187712482472,
        'n_filters': [32, 16, 12],
        'filter_sizes': [20, 10, 10],
        'output_filter_size': 20,
        'pool_sizes': [2, 2, 3],
        'delta': config.w,
        'batch_size': 92,
        'epochs': 10,
        'val_pct': 0.2
    }, 
    allow_val_change=True)

In [ ]:
%nbdev_hide
test.all_equal([len(x) for x in [config.n_filters, config.filter_sizes, config.pool_sizes]], np.repeat(len(config.n_filters), 3))

True

### Create the model

The implementation of the DCAE is done using Keras.

In [ ]:
%nbdev_export
def createDCAE(w, d, delta, n_filters=[64,32,12], filter_sizes=[10,5,5], pool_sizes=[2,2,3], output_filter_size=10):
    "Create a Deep Convolutional Autoencoder for multivariate time series of `d` dimensions, \
    sliced with a window size of `w`. The parameter `delta` sets the number of latent features that will be \
    contained in the Dense layer of the network. The the number of features \
    maps (filters), the filter size and the pool size can also be adjusted."
    # Test that the parameterization of the model is correct
    # 1. n_filters, filter_sizes and pool_sizes have the same length
    assert test.all_equal([len(x) for x in [n_filters, filter_sizes, pool_sizes]], np.repeat(len(n_filters), 3))
    # 2. Test that the number of filters in the last convLayer is equal to the product of the pool sizes
    assert np.prod(pool_sizes) == n_filters[-1]
    # 3. Test that the product of pool sizes is a divisor of the window size
    assert w % np.prod(pool_sizes).all() == 0
    # Create the model
    model = Sequential()
    model.add(InputLayer(input_shape=(w,d)))
    for (i, x) in enumerate(n_filters):
        model.add(Conv1D(filters=n_filters[i], kernel_size=filter_sizes[i], activation='relu', padding='same'))
        model.add(MaxPool1D(pool_size=pool_sizes[i]))
    aux_shape = model.output_shape[1:]
    model.add(Flatten())
    model.add(Dense(units=np.prod(aux_shape), activation='linear', name='latent_features'))
    model.add(Reshape(target_shape=aux_shape))
    for i, x in reversed(list(enumerate(n_filters))):
        model.add(Conv1D(filters=n_filters[i], kernel_size=filter_sizes[i], activation='relu', padding='same'))
        model.add(UpSampling1D(size=pool_sizes[i]))
    model.add(Conv1D(filters=d, kernel_size=output_filter_size, activation='linear', padding='same'))
    return model

In [ ]:
m = createDCAE(config.w, input_data.shape[-1], config.delta, n_filters=config.n_filters, 
              filter_sizes=config.filter_sizes, pool_sizes=config.pool_sizes,
              output_filter_size=config.output_filter_size)
m.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d (Conv1D)              (None, 48, 32)            10272     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 24, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 24, 16)            5136      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 12, 16)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 12, 12)            1932      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 12)             0         
_________________________________________________________________
flatten (Flatten)            (None, 48)                0

In [ ]:
opt = Adam(learning_rate=config.lr)

In [ ]:
m.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_squared_error'])

In [ ]:
history = m.fit(x=input_data, y=input_data, batch_size=config.batch_size, 
      validation_split=config.val_pct, epochs=config.epochs, verbose=0, 
      callbacks=[WandbCallback(log_weights=True)])




To track the performance of this model fit, go to the project dashboard in Weights & Biases. The link is provided at the beginning of this notebook, after the execution of the function `wandb.init()'' 